# In Class 4 - Dario Placencio.

Consider data set HealthData.txt. We build a neural network to predict the number of ER visit, i.e. 'ernum'.

In [172]:
# Import the required libraries
import pandas as pd
import numpy as np
import math
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense

In [173]:
# Read the HealData.txt
dat = pd.read_table("HealthData.txt", sep=' ')

In [174]:
# read data and check types
dat = pd.read_csv("HealthData.txt", sep=" ")
print(dat)

       year  age  female  race  married  edu  income  msa  region  limitation  \
0      2010   28       0     1        1   14   67000    1       1           0   
1      2010   25       1     1        1   14   67000    1       1           0   
2      2010   51       0     6        1   17   91420    1       4           0   
3      2010   53       1     4        1   17   91420    1       4           0   
4      2010   33       0     1        1   12   19500    1       3           0   
...     ...  ...     ...   ...      ...  ...     ...  ...     ...         ...   
36976  2011   54       1     4        1   12   23000    1       3           1   
36977  2011   21       0     1        1    9    7000    1       2           0   
36978  2011   21       1     1        1   10    7000    1       2           0   
36979  2011   19       1     1        0   10    9858    1       2           0   
36980  2011   25       0     1        0   12   15000    1       2           1   

       chronic  smoke  indu

In [175]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dat['industry'] = le.fit_transform(dat.industry.values)
dat['industry'].value_counts()

0     11042
10     6155
5      3411
9      2954
4      2602
11     2492
3      1649
8      1435
13     1365
12     1362
6      1274
7       517
1       399
14      128
15      107
2        89
Name: industry, dtype: int64

In [176]:
# Fit a Poisson Neural Net
# convert to float
convert_dict = {'ernum': float,
                'age': float,
                'income': float,
                'industry': float,
                }
dat = dat.astype(convert_dict)

In [177]:
# creat new variable
dat['logincome'] = np.log(dat['income'])

### Split data training and test sets, using observations in year 2010 as training and observations in year 2011 as test. Show the number of observations in each set.

In [178]:
# Filter the data for year 2010 and year 2011
train = dat[dat['year'] == 2010]
test = dat[dat['year'] == 2011]

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# Print the number of observations in each set
print("Number of observations in the training set: ", len(train))
print("Number of observations in the test set: ", len(test))

Number of observations in the training set:  17847
Number of observations in the test set:  19134


In [179]:
# Split the data into x and y variables, and then into training and test sets

numVar = ['age', 'logincome']
catVar = ['industry']

xnum_train = train[numVar]
xcat_train = train[catVar]

x_train = [xnum_train, xcat_train]
y_train = train['ernum']

xnum_test = test[numVar]
xcat_test = test[catVar]

x_test = [xnum_test, xcat_test]
y_test = test['ernum']

### Fit a Poisson neural network using three inputs: age, income (in log scale), and industry. Start with tuning parameters: (1) embedding dimension = 1; (2) number of hidden layers = 1; (3) number of neurons = 2. Calculate test data MSE.

In [200]:
from keras.layers import Embedding
from keras.layers import concatenate
from keras.layers import Reshape

# define the NN structure
# define numerical inputs 
in_num_layer = Input(shape = (xnum_train.shape[1],))
in_cat_layer = Input(shape = (1,))

# define embedding layers
embed_size = 1 # embedding size
em_layer = Embedding(input_dim = len(np.unique(xcat_train)), output_dim = embed_size)(in_cat_layer)
em_layer = Reshape(target_shape=(embed_size,))(em_layer)

in_layers = [in_num_layer,in_cat_layer]

# concatenate emebedding layer and numeric input layer together
conc_all = concatenate([in_num_layer, em_layer])
dense1 = Dense(2, activation='relu')(conc_all)
mu = Dense(1, activation = 'exponential')(dense1)   


# define model
model = Model(inputs = in_layers, outputs = mu)
model.summary()    

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_7 (Embedding)        (None, 1, 1)         16          ['input_16[0][0]']               
                                                                                                  
 input_15 (InputLayer)          [(None, 2)]          0           []                               
                                                                                                  
 reshape_7 (Reshape)            (None, 1)            0           ['embedding_7[0][0]']            
                                                                                            

In [201]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [202]:
model.fit(x_train, y_train, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 0s - loss: 0.9453 - 468ms/epoch - 3ms/step
Epoch 2/100
140/140 - 0s - loss: 0.8507 - 86ms/epoch - 615us/step
Epoch 3/100
140/140 - 0s - loss: 0.7756 - 89ms/epoch - 633us/step
Epoch 4/100
140/140 - 0s - loss: 0.7153 - 89ms/epoch - 634us/step
Epoch 5/100
140/140 - 0s - loss: 0.6665 - 88ms/epoch - 631us/step
Epoch 6/100
140/140 - 0s - loss: 0.6267 - 88ms/epoch - 626us/step
Epoch 7/100
140/140 - 0s - loss: 0.5940 - 89ms/epoch - 637us/step
Epoch 8/100
140/140 - 0s - loss: 0.5670 - 90ms/epoch - 645us/step
Epoch 9/100
140/140 - 0s - loss: 0.5447 - 91ms/epoch - 652us/step
Epoch 10/100
140/140 - 0s - loss: 0.5261 - 92ms/epoch - 655us/step
Epoch 11/100
140/140 - 0s - loss: 0.5106 - 88ms/epoch - 632us/step
Epoch 12/100
140/140 - 0s - loss: 0.4977 - 88ms/epoch - 627us/step
Epoch 13/100
140/140 - 0s - loss: 0.4870 - 89ms/epoch - 637us/step
Epoch 14/100
140/140 - 0s - loss: 0.4780 - 87ms/epoch - 621us/step
Epoch 15/100
140/140 - 0s - loss: 0.4706 - 87ms/epoch - 624us/step
Epoch

In [203]:
# make prediction
#yhat = model.predict(xtest)
#print(yhat)
yhat = model.predict(x_test)
print(yhat)

598/598 [==============================] - 0s 577us/step
[[0.1516585 ]
 [0.1516585 ]
 [0.1516585 ]
 ...
 [0.1516585 ]
 [0.15165849]
 [0.15165849]]


In [204]:
# Calculate the mean absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, yhat)
print("Mean Absolute Error: ", mae)

Mean Absolute Error:  0.2878060443415417


In [205]:
# Save the MAE result in a dataframe, with the row name as "original"
maes = pd.DataFrame(mae, index = ['original'], columns = ['MAE'])
maes

,MAE
original,0.287806


### From 2, change embedding dimension to 2, does this improve test data MSE?

In [206]:
# define the NN structure
# define numerical inputs 
in_num_layer = Input(shape = (xnum_train.shape[1],))
in_cat_layer = Input(shape = (1,))

# define embedding layers
embed_size = 2 # embedding size
em_layer = Embedding(input_dim = len(np.unique(xcat_train)), output_dim = embed_size)(in_cat_layer)
em_layer = Reshape(target_shape=(embed_size,))(em_layer)

in_layers = [in_num_layer,in_cat_layer]

# concatenate emebedding layer and numeric input layer together
conc_all = concatenate([in_num_layer, em_layer])
dense1 = Dense(2, activation='relu')(conc_all)
mu = Dense(1, activation = 'exponential')(dense1)   


# define model
model = Model(inputs = in_layers, outputs = mu)
model.summary()   

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, 1, 2)         32          ['input_18[0][0]']               
                                                                                                  
 input_17 (InputLayer)          [(None, 2)]          0           []                               
                                                                                                  
 reshape_8 (Reshape)            (None, 2)            0           ['embedding_8[0][0]']            
                                                                                            

In [207]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [208]:
model.fit(x_train, y_train, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 0s - loss: 2.1240 - 466ms/epoch - 3ms/step
Epoch 2/100
140/140 - 0s - loss: 1.6058 - 87ms/epoch - 623us/step
Epoch 3/100
140/140 - 0s - loss: 0.9715 - 88ms/epoch - 627us/step
Epoch 4/100
140/140 - 0s - loss: 0.7525 - 88ms/epoch - 631us/step
Epoch 5/100
140/140 - 0s - loss: 0.6652 - 88ms/epoch - 628us/step
Epoch 6/100
140/140 - 0s - loss: 0.6116 - 88ms/epoch - 628us/step
Epoch 7/100
140/140 - 0s - loss: 0.5730 - 88ms/epoch - 632us/step
Epoch 8/100
140/140 - 0s - loss: 0.5435 - 88ms/epoch - 630us/step
Epoch 9/100
140/140 - 0s - loss: 0.5206 - 88ms/epoch - 631us/step
Epoch 10/100
140/140 - 0s - loss: 0.5025 - 99ms/epoch - 709us/step
Epoch 11/100
140/140 - 0s - loss: 0.4880 - 90ms/epoch - 642us/step
Epoch 12/100
140/140 - 0s - loss: 0.4766 - 87ms/epoch - 620us/step
Epoch 13/100
140/140 - 0s - loss: 0.4676 - 86ms/epoch - 616us/step
Epoch 14/100
140/140 - 0s - loss: 0.4603 - 88ms/epoch - 627us/step
Epoch 15/100
140/140 - 0s - loss: 0.4546 - 86ms/epoch - 617us/step
Epoch

In [209]:
# Make prediction
yhat = model.predict(x_test)
print(yhat)

598/598 [==============================] - 0s 601us/step
[[0.11689119]
 [0.2222968 ]
 [0.12347271]
 ...
 [0.14083034]
 [0.14030015]
 [0.16066   ]]


In [210]:
# Calculate the mean absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, yhat)
print("Mean Absolute Error: ", mae)

Mean Absolute Error:  0.2862252850072647


In [211]:
# Append the MAE result to the maes dataframe, with the row name as "embed_size = 2"
maes.loc['embed_size = 2'] = mae
maes

,MAE
original,0.287806
embed_size = 2,0.286225


The added embedding dimension improve slightly the test data MSE.

### From 2, change number of neurons to 5, does this improve test MSE?

In [212]:
# define the NN structure
# define numerical inputs 
in_num_layer = Input(shape = (xnum_train.shape[1],))
in_cat_layer = Input(shape = (1,))

# define embedding layers
embed_size = 1 # embedding size
em_layer = Embedding(input_dim = len(np.unique(xcat_train)), output_dim = embed_size)(in_cat_layer)
em_layer = Reshape(target_shape=(embed_size,))(em_layer)

in_layers = [in_num_layer,in_cat_layer]

# concatenate emebedding layer and numeric input layer together
conc_all = concatenate([in_num_layer, em_layer])
dense1 = Dense(5, activation='relu')(conc_all)
mu = Dense(1, activation = 'exponential')(dense1)   


# define model
model = Model(inputs = in_layers, outputs = mu)
model.summary()    

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_9 (Embedding)        (None, 1, 1)         16          ['input_20[0][0]']               
                                                                                                  
 input_19 (InputLayer)          [(None, 2)]          0           []                               
                                                                                                  
 reshape_9 (Reshape)            (None, 1)            0           ['embedding_9[0][0]']            
                                                                                            

In [213]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [214]:
model.fit(x_train, y_train, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 0s - loss: 2035.4486 - 452ms/epoch - 3ms/step
Epoch 2/100
140/140 - 0s - loss: 167.2197 - 88ms/epoch - 627us/step
Epoch 3/100
140/140 - 0s - loss: 65.0640 - 91ms/epoch - 649us/step
Epoch 4/100
140/140 - 0s - loss: 28.4027 - 91ms/epoch - 650us/step
Epoch 5/100
140/140 - 0s - loss: 15.7059 - 90ms/epoch - 642us/step
Epoch 6/100
140/140 - 0s - loss: 10.0280 - 91ms/epoch - 649us/step
Epoch 7/100
140/140 - 0s - loss: 7.0159 - 88ms/epoch - 627us/step
Epoch 8/100
140/140 - 0s - loss: 5.2408 - 89ms/epoch - 639us/step
Epoch 9/100
140/140 - 0s - loss: 4.1164 - 90ms/epoch - 641us/step
Epoch 10/100
140/140 - 0s - loss: 3.3658 - 93ms/epoch - 663us/step
Epoch 11/100
140/140 - 0s - loss: 2.8445 - 98ms/epoch - 703us/step
Epoch 12/100
140/140 - 0s - loss: 2.4710 - 88ms/epoch - 630us/step
Epoch 13/100
140/140 - 0s - loss: 2.1970 - 87ms/epoch - 624us/step
Epoch 14/100
140/140 - 0s - loss: 1.9920 - 87ms/epoch - 624us/step
Epoch 15/100
140/140 - 0s - loss: 1.8361 - 88ms/epoch - 631us/s

In [215]:
# make prediction
#yhat = model.predict(xtrain)
#print(yhat)
xnum_new = pd.DataFrame(data = [[3,5]], columns = ['age','logincome'])
xcat_new = pd.DataFrame(data = [3], columns = ['industry'])
x_new = [xnum_new,xcat_new]
yhat = model.predict(x_new)
print(yhat)

1/1 [==============================] - 0s 52ms/step
[[0.09896413]]


In [216]:
# Make prediction
yhat = model.predict(x_test)
print(yhat)

598/598 [==============================] - 0s 544us/step
[[0.10438475]
 [0.18647037]
 [0.14790265]
 ...
 [0.10715509]
 [0.12057173]
 [0.12017579]]


In [217]:
# Calculate the mean absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, yhat)
print("Mean Absolute Error: ", mae)

Mean Absolute Error:  0.28814282098239313


In [218]:
# Append the MAE result to the maes dataframe, with the row name as "dense_size = 5"
maes.loc['dense_size = 5'] = mae
maes

,MAE
original,0.287806
embed_size = 2,0.286225
dense_size = 5,0.288143


The added number of neurons does not improve the test data MSE.

### From 2, change number of hidden layers to 2, does this improve test MSE?

In [225]:
# define the NN structure
# define numerical inputs 
in_num_layer = Input(shape = (xnum_train.shape[1],))
in_cat_layer = Input(shape = (1,))

# define embedding layers
embed_size = 1 # embedding size
em_layer = Embedding(input_dim = len(np.unique(xcat_train)), output_dim = embed_size)(in_cat_layer)
em_layer = Reshape(target_shape=(embed_size,))(em_layer)

in_layers = [in_num_layer,in_cat_layer]

# concatenate emebedding layer and numeric input layer together
conc_all = concatenate([in_num_layer, em_layer])
dense1 = Dense(2, activation='relu')(conc_all)
dense2 = Dense(2, activation='relu')(dense1)
mu = Dense(1, activation = 'exponential')(dense2)   

# define model
model = Model(inputs = in_layers, outputs = mu)
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_24 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding_11 (Embedding)       (None, 1, 1)         16          ['input_24[0][0]']               
                                                                                                  
 input_23 (InputLayer)          [(None, 2)]          0           []                               
                                                                                                  
 reshape_11 (Reshape)           (None, 1)            0           ['embedding_11[0][0]']           
                                                                                           

In [226]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [227]:
model.fit(x_train, y_train, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 0s - loss: 0.9453 - 499ms/epoch - 4ms/step
Epoch 2/100
140/140 - 0s - loss: 0.8506 - 94ms/epoch - 669us/step
Epoch 3/100
140/140 - 0s - loss: 0.7756 - 96ms/epoch - 682us/step
Epoch 4/100
140/140 - 0s - loss: 0.7153 - 96ms/epoch - 683us/step
Epoch 5/100
140/140 - 0s - loss: 0.6665 - 98ms/epoch - 704us/step
Epoch 6/100
140/140 - 0s - loss: 0.6267 - 97ms/epoch - 696us/step
Epoch 7/100
140/140 - 0s - loss: 0.5940 - 97ms/epoch - 693us/step
Epoch 8/100
140/140 - 0s - loss: 0.5670 - 97ms/epoch - 695us/step
Epoch 9/100
140/140 - 0s - loss: 0.5447 - 96ms/epoch - 682us/step
Epoch 10/100
140/140 - 0s - loss: 0.5261 - 95ms/epoch - 677us/step
Epoch 11/100
140/140 - 0s - loss: 0.5106 - 95ms/epoch - 681us/step
Epoch 12/100
140/140 - 0s - loss: 0.4977 - 92ms/epoch - 658us/step
Epoch 13/100
140/140 - 0s - loss: 0.4870 - 107ms/epoch - 765us/step
Epoch 14/100
140/140 - 0s - loss: 0.4780 - 93ms/epoch - 663us/step
Epoch 15/100
140/140 - 0s - loss: 0.4706 - 90ms/epoch - 645us/step
Epoc

In [228]:
# Make prediction
yhat = model.predict(x_test)
print(yhat)

598/598 [==============================] - 0s 611us/step
[[0.15170106]
 [0.15170106]
 [0.15170106]
 ...
 [0.15170106]
 [0.15170106]
 [0.15170106]]


In [229]:
# Calculate the mean absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, yhat)
print("Mean Absolute Error: ", mae)

Mean Absolute Error:  0.28782066853925875


In [230]:
# Append the MAE result to the maes dataframe, with the row name as "Additional Layer"
maes.loc['Additional Layer'] = mae
maes

,MAE
original,0.287806
embed_size = 2,0.286225
dense_size = 5,0.288143
Additional Layer,0.287821


The added number of hidden layers does not improve the test data MSE.